In [9]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from IPython.display import display, HTML
import pandas as pd
from src.data_loader import load_local_data, unload_local_data

bronze_data = load_local_data()
new_ads = bronze_data[['site', 'job_title', 'area', 'work_location','ingestion_ts', 'link']].sort_values(by='ingestion_ts', ascending=False)
new_ads["title_link"] = new_ads.apply(
        lambda x: f'<a href="{x["link"]}" target="_blank">{x["job_title"]}</a>', axis=1
    )

In [10]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Copy of job ads
df = new_ads.copy()

# Ensure you have the clickable link column
df['title_link'] = df.apply(
    lambda x: f'<a href="{x["link"]}" target="_blank">{x["job_title"]}</a>', axis=1
)

# Predefined keywords
keywords_list = ["Analytiker", "Analys", "Bi", "Data", "ML", "Machine Learning", "Business Intelligence"]

# Create checkbox per keyword
checkboxes = [widgets.Checkbox(value=False, description=kw) for kw in keywords_list]

# Text-widget for custom keyword
custom_keyword = widgets.Text(
    value='',
    placeholder='Skriv eget keyword här',
    description='Eget keyword:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Output area
output = widgets.Output()

# Function to render the HTML table with clickable job_title
def return_styled_jobs_table(data):
    df_display = data[["site", "title_link", "work_location", "ingestion_ts"]]
    html_table = df_display.to_html(escape=False, index=False)
    styled_html = f"""
    <style>
        td:nth-child(2), th:nth-child(2) {{
            text-align: left !important;
        }}
    </style>
    {html_table}
    """
    return HTML(styled_html)

# Filtering function
def filter_table(change=None):
    with output:
        clear_output()
        # Selected keywords from checkboxes
        selected = [cb.description for cb in checkboxes if cb.value]
        # Add custom keyword if present
        if custom_keyword.value.strip():
            selected.append(custom_keyword.value.strip())
        
        if not selected:
            display(return_styled_jobs_table(df))
        else:
            mask = df['job_title'].str.contains('|'.join(selected), case=False, na=False)
            display(return_styled_jobs_table(df[mask]))

# Connect onchange events
for cb in checkboxes:
    cb.observe(filter_table, 'value')
custom_keyword.observe(filter_table, 'value')

# Display controls
controls = widgets.HBox(checkboxes + [custom_keyword])
display(controls, output)

# Display full table initially
filter_table()


Output()